In [ ]:
p1 = "Doch das Ende des Jahres 2020 birgt auch Hoffnung, dass durch die Vakzinen \
gegen Covid-19 wieder Normalität einkehre – wie immer die auch aussehen mag \
– und wir uns um anderes Dringliches kümmern oder einfach entspannen \
können. Und dass durch den im Januar anstehenden Bewohnerwechsel im \
Weißen Haus zu Washington D.C. das offizielle Herumgetrumpel auf dem \
gesunden Menschenverstand ein Ende finden möge."

p2 = "Wir, das gesamte Team von heise online und die Redaktionen von c't, iX, \
Technology Review, Mac & i, c't Digitale Fotografie, Make:, Techstage und \
Telepolis sowie heise Security, heise Developer und heise Autos wünschen Ihnen \
ein friedliches und freudvolles Jahr 2021. Wir wünschen Ihnen, dass Sie nicht \
vergeblich hoffen und dass Ihre Vorsätze erfüllt werden, auf dass Sie gesund \
bleiben oder genesen."

In [ ]:
!pip install textacy
!python -m spacy download de_core_news_lg

In [ ]:
import spacy

nlp = spacy.load("de_core_news_lg")
d1 = nlp(p1)
d2 = nlp(p2)

In [ ]:
import pandas as pd

pd.set_option("max.rows", None)
pd.set_option('display.max_colwidth', None)

def tag_morph(d):
    res = []
    for token in d:
        res.append([token.text, token.tag_, token.pos_, str(token.morph)])

    return pd.DataFrame(res, columns=["Text", "Tag", "POS", "Morphologie"])

In [ ]:
tm1 = tag_morph(d1)
tm1

In [ ]:
# Substantive
tm1[(tm1["POS"] == "NOUN") | (tm1["POS"] == "PROPN")]["Tag"].unique()

In [ ]:
# Adjektive
tm1[(tm1["POS"] == "ADJ") | (tm1["POS"] == "ADV")]["Tag"].unique()

In [ ]:
# Verben
tm1[(tm1["POS"] == "VERB") | (tm1["POS"] == "AUX")]["Tag"].unique()

In [ ]:
def word_types(doc):
    nouns = []
    adjectives = []
    verbs = []
    lemmas = []
    nav = []
    for token in doc:
        lemmas.append(token.lemma_)
        # adjective
        if token.pos_ == "ADJ" or token.pos_ == "ADV":
            adjectives.append(token.lemma_)
            nav.append(token.lemma_)
        # noun
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            nouns.append(token.lemma_)
            nav.append(token.lemma_)
        # verb
        if token.pos_ == "VERB":
            verbs.append(token.lemma_)
            nav.append(token.lemma_)
            
    return (nouns, adjectives, verbs, nav, lemmas)

In [ ]:
(nouns, adjectives, verbs, nav, lemmas) = word_types(d1)
print(f'Substantive: {"|".join(nouns)}')
print(f'Adjektive: {"|".join(adjectives)}')
print(f'Verben: {"|".join(verbs)}')
print(f'Kombinationen: {"|".join(nav)}')
print(f'Lemmas: {"|".join(lemmas)}')